In [1]:
!pip install pymysql pandas tqdm python-dotenv

In [1]:
import pandas as pd
import pymysql
from tqdm import tqdm
from dotenv import load_dotenv
import os

In [2]:
df = pd.read_csv('./data/v2_temp_csv.csv')

In [3]:
load_dotenv()

True

In [5]:
conn = pymysql.connect(
    host=os.getenv('MYSQL_HOST'),
    user=os.getenv('MYSQL_USER'),
    password=os.getenv('MYSQL_PASSWORD'),
    db=os.getenv('MYSQL_DATABASE'),
    port=int(os.getenv('MYSQL_PORT')),
    charset='utf8'
)

In [7]:
df['st_id'].value_counts()

st_id
48    604800
Name: count, dtype: int64

In [11]:
df['weekday'].value_counts()

weekday
Wednesday    86400
Thursday     86400
Friday       86400
Saturday     86400
Sunday       86400
Monday       86400
Tuesday      86400
Name: count, dtype: int64

In [13]:
cursor = conn.cursor()

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 604800 entries, 0 to 604799
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   dtime             604800 non-null  object 
 1   Temp              604800 non-null  float64
 2   Humidity          604800 non-null  float64
 3   AtmosphericPress  604800 non-null  float64
 4   PM1               604800 non-null  int64  
 5   PM2_5             604800 non-null  int64  
 6   PM10              604800 non-null  int64  
 7   CODen             604800 non-null  float64
 8   CO2Den            604800 non-null  int64  
 9   weekday           604800 non-null  object 
 10  time_hms          604800 non-null  object 
 11  st_id             604800 non-null  int64  
dtypes: float64(4), int64(5), object(3)
memory usage: 55.4+ MB


In [17]:
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [ ]:
for day in weekdays:
    day_df = df[df['weekday'] == day]
    for i, row in tqdm(day_df.iterrows(), total=len(day_df), desc=day):
        sql = """
        INSERT INTO sensor (
            dtime, time_hms, weekday, temp, humidity, atmospheric_press,
            pm1, pm25, pm10, coden, co2den, st_id
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        """
        cursor.execute(sql, (
            row['dtime'], row['time_hms'], row['weekday'], row['Temp'], row['Humidity'], row['AtmosphericPress'],
            row['PM1'], row['PM2_5'], row['PM10'], row['CODen'], row['CO2Den'], row['st_id']
        ))
    conn.commit()


Monday:  11%|███████▎                                                             | 9091/86400 [03:44<27:41, 46.53it/s]

In [ ]:
cursor.close()
conn.close()